In [5]:
import sys
sys.path.append('./KPConv-PyTorch/')

In [25]:
from utils.config import Config
from models.architectures import KPFCNN
import numpy as np

In [15]:
class CellDataConfig(Config):
    """
    Override the parameters you want to modify for this dataset
    """

    ####################
    # Dataset parameters
    ####################

    # Dataset name
    dataset = 'S3DIS'

    # Number of classes in the dataset (This value is overwritten by dataset class when Initializating dataset).
    num_classes = None

    # Type of task performed on this dataset (also overwritten)
    dataset_task = ''

    # Number of CPU threads for the input pipeline
    input_threads = 10

    #########################
    # Architecture definition
    #########################

    # Define layers
    architecture = ['simple',
                    'resnetb',
                    'unary']

    ###################
    # KPConv parameters
    ###################

    # Radius of the input sphere
    in_radius = 1.5

    # Number of kernel points
    num_kernel_points = 15

    # Size of the first subsampling grid in meter
    first_subsampling_dl = 0.03

    # Radius of convolution in "number grid cell". (2.5 is the standard value)
    conv_radius = 2.5

    # Radius of deformable convolution in "number grid cell". Larger so that deformed kernel can spread out
    deform_radius = 6.0

    # Radius of the area of influence of each kernel point in "number grid cell". (1.0 is the standard value)
    KP_extent = 1.2

    # Behavior of convolutions in ('constant', 'linear', 'gaussian')
    KP_influence = 'linear'

    # Aggregation function of KPConv in ('closest', 'sum')
    aggregation_mode = 'sum'

    # Choice of input features
    first_features_dim = 128
    in_features_dim = 4

    # Can the network learn modulations
    modulated = False

    # Batch normalization parameters
    use_batch_norm = True
    batch_norm_momentum = 0.02

    # Deformable offset loss
    # 'point2point' fitting geometry by penalizing distance from deform point to input points
    # 'point2plane' fitting geometry by penalizing distance from deform point to input point triplet (not implemented)
    deform_fitting_mode = 'point2point'
    deform_fitting_power = 1.0              # Multiplier for the fitting/repulsive loss
    deform_lr_factor = 0.1                  # Multiplier for learning rate applied to the deformations
    repulse_extent = 1.2                    # Distance of repulsion for deformed kernel points

    #####################
    # Training parameters
    #####################

    # Maximal number of epochs
    max_epoch = 500

    # Learning rate management
    learning_rate = 1e-2
    momentum = 0.98
    lr_decays = {i: 0.1 ** (1 / 150) for i in range(1, max_epoch)}
    grad_clip_norm = 100.0

    # Number of batch
    batch_num = 6

    # Number of steps per epochs
    epoch_steps = 500

    # Number of validation examples per epoch
    validation_size = 50

    # Number of epoch between each checkpoint
    checkpoint_gap = 50

    # Augmentations
    augment_scale_anisotropic = True
    augment_symmetries = [True, False, False]
    augment_rotation = 'vertical'
    augment_scale_min = 0.8
    augment_scale_max = 1.2
    augment_noise = 0.001
    augment_color = 0.8

    # The way we balance segmentation loss
    #   > 'none': Each point in the whole batch has the same contribution.
    #   > 'class': Each class has the same contribution (points are weighted according to class balance)
    #   > 'batch': Each cloud in the batch has the same contribution (points are weighted according cloud sizes)
    segloss_balance = 'none'

    # Do we nee to save convergence
    saving = True
    saving_path = None

In [16]:
config = CellDataConfig()
model = KPFCNN(config, [0,1], [])

In [17]:
model

KPFCNN(
  (encoder_blocks): ModuleList(
    (0): SimpleBlock(
      (KPConv): KPConv(radius: 0.07, in_feat: 4, out_feat: 64)
      (batch_norm): BatchNormBlock(in_feat: 64, momentum: 0.020, only_bias: False)
      (leaky_relu): LeakyReLU(negative_slope=0.1)
    )
    (1): ResnetBottleneckBlock(
      (unary1): UnaryBlock(in_feat: 64, out_feat: 32, BN: True, ReLU: True)
      (KPConv): KPConv(radius: 0.07, in_feat: 32, out_feat: 32)
      (batch_norm_conv): BatchNormBlock(in_feat: 32, momentum: 0.020, only_bias: False)
      (unary2): UnaryBlock(in_feat: 32, out_feat: 128, BN: True, ReLU: False)
      (unary_shortcut): UnaryBlock(in_feat: 64, out_feat: 128, BN: True, ReLU: False)
      (leaky_relu): LeakyReLU(negative_slope=0.1)
    )
    (2): UnaryBlock(in_feat: 128, out_feat: 128, BN: True, ReLU: True)
  )
  (decoder_blocks): ModuleList(
    (0): SimpleBlock(
      (KPConv): KPConv(radius: 0.07, in_feat: 128, out_feat: 64)
      (batch_norm): BatchNormBlock(in_feat: 64, momentum: 0.02

In [26]:
class data():
    def __init__(self, input_data = []):
        self.features = input_data[-1]
        self.points = input_data[:3]

test_data = np.array([[0, 0, 0, 256],
                     [0, 1, 1, 0]])
test_input = data(input_data=test_data)

In [27]:
model(test_input, config)

AttributeError: 'numpy.ndarray' object has no attribute 'clone'